### Setup Instructions

Please follow the [setup instructions](/docs/cv-tutorial-setup) to prepare your environment if you haven't yet.   
This tutorial will be referencing this [Notebook](https://github.com/outerbounds/tutorials/blob/main/cv/cv-intro-2.ipynb).

Now it’s time to build a model to compare against the baseline. We are going to define a [Convolutional Neural Network](https://en.wikipedia.org/wiki/Convolutional_neural_network) model.

The model operates directly on images, so there is no need to flatten the image data like in the case of the feed-forward neural network.

In [1]:
from tensorflow import keras
import numpy as np

num_classes = 10
((x_train, y_train), (x_test, y_test)) = keras.datasets.mnist.load_data()
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

The model has several hidden layers defined by the `hidden_conv_layer_sizes` hyperparameter. 
You don't need to know about these or change them for now.

In [2]:
hidden_conv_layer_sizes = [32, 64]
input_shape = (28, 28, 1)
kernel_size = (3, 3)
pool_size = (2, 2)
p_dropout = 0.5

num_classes = y_test.shape[1]

epochs = 5
batch_size = 32
verbose = 2 

optimizer = 'adam'
loss = 'categorical_crossentropy'
metrics = ['accuracy']

This `hidden_conv_layer_sizes` list determines the number hidden layers and the of output dimensions in each hidden convolutional layer. 

In [3]:
from tensorflow.keras import layers, Sequential, Input

_layers = [Input(shape=input_shape)]

# dynamic based on length of hidden_conv_layer_sizes
for conv_layer_size in hidden_conv_layer_sizes:
    _layers.append(layers.Conv2D(
        conv_layer_size, 
        kernel_size=kernel_size, 
        activation="relu"
    ))
    _layers.append(layers.MaxPooling2D(pool_size=pool_size))
_layers.extend([
    layers.Flatten(),
    layers.Dropout(p_dropout),
    layers.Dense(num_classes, activation="softmax")
])

model = Sequential(_layers)
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

Keras models like the one you made in the previous step have a `.fit` function following the [sklearn Estimator API](https://scikit-learn.org/stable/developers/develop.html).

In [4]:
history = model.fit(
    x_train, y_train,
    validation_data = (x_test, y_test),
    epochs = epochs,
    batch_size = batch_size,
    verbose = verbose
)

Epoch 1/5


2022-10-11 17:15:51.289621: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1875/1875 - 11s - loss: 0.2057 - accuracy: 0.9365 - val_loss: 0.0581 - val_accuracy: 0.9805 - 11s/epoch - 6ms/step
Epoch 2/5
1875/1875 - 11s - loss: 0.0769 - accuracy: 0.9760 - val_loss: 0.0427 - val_accuracy: 0.9859 - 11s/epoch - 6ms/step
Epoch 3/5
1875/1875 - 12s - loss: 0.0621 - accuracy: 0.9809 - val_loss: 0.0352 - val_accuracy: 0.9878 - 12s/epoch - 7ms/step
Epoch 4/5
1875/1875 - 13s - loss: 0.0526 - accuracy: 0.9834 - val_loss: 0.0309 - val_accuracy: 0.9893 - 13s/epoch - 7ms/step
Epoch 5/5
1875/1875 - 13s - loss: 0.0465 - accuracy: 0.9853 - val_loss: 0.0270 - val_accuracy: 0.9900 - 13s/epoch - 7ms/step


Keras models also have a `.evaluate` function you can use to retrieve accuracy and loss scores.

In [5]:
scores = model.evaluate(x_test, y_test, verbose=0)
categorical_cross_entropy = scores[0]
accuracy = scores[1]
msg = "The CNN model predicted correctly {}% of the time on the test set."
print(msg.format(round(100*accuracy, 3)))

The CNN model predicted correctly 99.0% of the time on the test set.
